In [1]:
!pip uninstall langchain langchain-core langchain-community langchain-openai -y
!pip install langchain==0.3.10 langchain-openai langchain-community faiss-cpu pydantic

Found existing installation: langchain 1.1.3
Uninstalling langchain-1.1.3:
  Successfully uninstalled langchain-1.1.3
Found existing installation: langchain-core 1.1.3
Uninstalling langchain-core-1.1.3:
  Successfully uninstalled langchain-core-1.1.3
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to 

In [1]:
!pip show langchain


Name: langchain
Version: 0.3.10
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


In [2]:
import os
from google.colab import userdata

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_community.vectorstores import FAISS
import faiss

# from langchain_community.vectorstores import FAISS # Already imported
from langchain.tools.retriever import create_retriever_tool
from langchain_core.documents import Document


# from langsmith import Client
# client = Client(api_key=userdata.get('LANGSMITH_API_KEY'))
# prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)
# print(prompt)


In [11]:
import os
from google.colab import userdata

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_community.vectorstores import FAISS
import faiss

from langchain.tools.retriever import create_retriever_tool
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage

# CẤU HÌNH API KEY
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

# Định nghĩa biến chung cho URL API
API_BASE_URL = "https://aiportalapi.stu-platform.live/jpe"

# ==============================================================================
# PHẦN 1: XÂY DỰNG RAG (KNOWLEDGE BASE)
# ==============================================================================

# Giả lập dữ liệu kiến thức (Trong thực tế bạn sẽ load từ PDF/Text)
rag_docs = [
    Document(page_content="Sự khác biệt giữa Affect và Effect: 'Affect' thường là động từ (tác động), 'Effect' thường là danh từ (kết quả/hiệu ứng). Ví dụ: The rain affected my mood. The effect of the rain was sadness.", metadata={"topic": "grammar_diff"}),
    Document(page_content="Thành ngữ 'Break a leg' có nghĩa là 'Chúc may mắn', thường dùng trước khi ai đó lên sân khấu biểu diễn.", metadata={"topic": "idioms"}),
    Document(page_content="Thì hiện tại hoàn thành (Present Perfect) dùng để diễn tả hành động xảy ra trong quá khứ nhưng kết quả còn lưu lại ở hiện tại hoặc thời gian không xác định.", metadata={"topic": "grammar_rules"}),
    Document(page_content="Anh-Anh (BrE) dùng 'Colour', trong khi Anh-Mỹ (AmE) dùng 'Color'. Tương tự với Centre (BrE) và Center (AmE).", metadata={"topic": "culture"}),
]


# Tạo Vector Store và Retriever
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              base_url=API_BASE_URL,
                              api_key=userdata.get('TEXT_EMBEDDING_3_SMALL_KEY'))
vector = FAISS.from_documents(rag_docs, embeddings)
retriever = vector.as_retriever()

# Biến Retriever thành một Tool để Agent có thể gọi
retriever_tool = create_retriever_tool(
    retriever,
    name="search_english_knowledge",
    description="Sử dụng công cụ này KHI VÀ CHỈ KHI người dùng hỏi về kiến thức tiếng Anh, thành ngữ, sự khác biệt từ vựng hoặc văn hóa. KHÔNG dùng để sửa lỗi ngữ pháp."
)

# ==============================================================================
# PHẦN 2: XÂY DỰNG FUNCTION TOOLS
# ==============================================================================

# Tool 1: Sửa lỗi ngữ pháp
@tool
def grammar_corrector(text_to_correct: str) -> str:
    """
    Dùng để sửa lỗi ngữ pháp tiếng Anh một cách tự động sử dụng LLM.
    Input là đoạn văn bản gốc cần sửa.
    Output là văn bản đã sửa và giải thích ngắn gọn.
    """
    grammar_llm = ChatOpenAI(
        model="GPT-4o-mini",
        base_url=API_BASE_URL,
        api_key=userdata.get('OPENAI_KEY'),
        temperature=0.2
    )

    correction_prompt = ChatPromptTemplate.from_messages([
        ("system", "Bạn là một trợ lý sửa lỗi ngữ pháp tiếng Anh chuyên nghiệp. Nhiệm vụ của bạn là sửa các lỗi ngữ pháp, chính tả, dấu câu trong văn bản của người dùng, và cung cấp một giải thích ngắn gọn, rõ ràng về lý do sửa lỗi đó. Hãy luôn giữ định dạng đầu ra như sau:\nCorrected: [Văn bản đã sửa]\nExplanation: [Giải thích lỗi và cách sửa]"),
        ("human", "{text_to_correct}"),
    ])

    correction_chain = correction_prompt | grammar_llm

    response = correction_chain.invoke({"text_to_correct": text_to_correct})
    llm_output = response.content

    corrected_text = ""
    explanation = ""

    lines = llm_output.split('\n')
    for line in lines:
        if line.startswith("Corrected:"):
            corrected_text = line.replace("Corrected:", "").strip()
        elif line.startswith("Explanation:"):
            explanation = line.replace("Explanation:", "").strip()

    if not corrected_text and not explanation:
        corrected_text = llm_output
        explanation = "Không thể phân tích định dạng đầu ra từ LLM hoặc không có giải thích cụ thể."
    elif not corrected_text and explanation:
        corrected_text = text_to_correct
    elif corrected_text and not explanation:
        explanation = "Không có giải thích cụ thể từ LLM."


    return f"""
    [CORRECTION REPORT]
    Original: {text_to_correct}
    Corrected: {corrected_text}
    Explanation: {explanation}
    """

@tool
def learning_suggestion_generator(current_level: str = None, goal: str = None) -> str:
    """
    Dùng để tạo gợi ý lộ trình học tập.
    Input yêu cầu 'current_level' (ví dụ: Beginner, A1, B1, C1) và 'goal' (ví dụ: IELTS 7.0, giao tiếp tự tin, đọc sách không cần từ điển).
    Nếu một trong các tham số bị thiếu, công cụ sẽ cung cấp phản hồi phù hợp hoặc gợi ý chung.
    """
    if current_level is None and goal is None:
        return "Để tôi có thể đưa ra gợi ý học tập tốt nhất, bạn vui lòng cho biết trình độ tiếng Anh hiện tại của bạn (ví dụ: Beginner, A1, B1, C1) và mục tiêu học tập của bạn là gì (ví dụ: IELTS 7.0, giao tiếp tự tin, đọc sách không cần từ điển)."
    elif current_level is None:
        return f"Bạn muốn đạt mục tiêu '{goal}' nhưng tôi chưa biết trình độ tiếng Anh hiện tại của bạn. Vui lòng cung cấp trình độ hiện tại (ví dụ: Beginner, A1, B1, C1) để tôi có thể đưa ra gợi ý phù hợp."
    elif goal is None:
        return f"Với trình độ '{current_level}' của bạn, bạn muốn đạt được mục tiêu gì? (ví dụ: IELTS 7.0, giao tiếp tự tin, đọc sách không cần từ điển) để tôi có thể đưa ra lộ trình cụ thể."

    suggestions = f"Dựa trên trình độ {current_level} và mục tiêu {goal}, đây là lộ trình gợi ý:\n"
    if "IELTS" in goal.upper():
        suggestions += "- Tập trung vào từ vựng Academic (Cambridge Vocab for IELTS).\n- Luyện Writing Task 1 hàng ngày."
    elif "GIAO TIẾP" in goal.upper() or "COMMUNICATION" in goal.upper():
        suggestions += "- Tập trung vào Shadowing để luyện phát âm.\n- Học qua phim Friends hoặc Podcast BBC Learning English."
    elif "ĐỌC SÁCH" in goal.upper() or "READING" in goal.upper():
        suggestions += "- Bắt đầu với sách truyện đơn giản, dần dần tăng độ khó.\n- Đọc các bài báo hoặc blog về chủ đề bạn yêu thích."
    else:
        suggestions += "- Tìm kiếm tài liệu và khóa học phù hợp với mục tiêu của bạn.\n- Luyện tập thường xuyên các kỹ năng (Nghe, Nói, Đọc, Viết) để nâng cao trình độ."

    return suggestions

# ==============================================================================
# PHẦN 3: KHỞI TẠO AGENT
# ==============================================================================

# Tập hợp các tools
tools = [grammar_corrector, learning_suggestion_generator, retriever_tool]

# Khởi tạo LLM (Brain của Agent)
llm = ChatOpenAI(model="GPT-4o-mini", base_url=API_BASE_URL, temperature=0)

# Tạo Prompt cho Agent (System Prompt rất quan trọng để định hướng)
prompt = ChatPromptTemplate.from_messages([
    ("system", f"Bạn là một trợ lý học tiếng Anh thông minh. Bạn có các công cụ để sửa lỗi, gợi ý học tập và tra cứu kiến thức."
                "Hãy chọn công cụ phù hợp nhất dựa trên yêu cầu của người dùng."),
    MessagesPlaceholder(variable_name="chat_history"), # Thêm placeholder cho lịch sử trò chuyện
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"), # Nơi Agent suy nghĩ và gọi tool
])

# Tạo Agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ==============================================================================
# PHẦN 4: CHẠY THỬ NGHIỆM (SCENARIOS)
# ==============================================================================

def interactive_chat():
    print("Chào mừng bạn đến với trợ lý học tiếng Anh!\n")
    print("Hãy nhập câu hỏi hoặc yêu cầu của bạn. Gõ 'quit' hoặc 'exit' để dừng trò chuyện.\n")
    chat_history = []
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", ""]:
            print("Agent: Tạm biệt! Chúc bạn học tốt!")
            break
        print("-" * 100)
        response = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
        print(f"Agent: {response['output']}")
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response['output']))
        print("-" * 100)

interactive_chat()


Chào mừng bạn đến với trợ lý học tiếng Anh!

Hãy nhập câu hỏi hoặc yêu cầu của bạn. Gõ 'quit' hoặc 'exit' để dừng trò chuyện.

User: Sự khác biệt giữa Affect và Effect
----------------------------------------------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `search_english_knowledge` with `{'query': 'difference between affect and effect'}`


Sự khác biệt giữa Affect và Effect: 'Affect' thường là danh từ (kết quả/hiệu ứng), 'Effect' thường là động từ (tác động). Ví dụ: The rain affected my mood. The effect of the rain was sadness.

Anh-Anh (BrE) dùng 'Colour', trong khi Anh-Mỹ (AmE) dùng 'Color'. Tương tự với Centre (BrE) và Center (AmE).

Thì hiện tại hoàn thành (Present Perfect) dùng để diễn tả hành động xảy ra trong quá khứ nhưng kết quả còn lưu lại ở hiện tại hoặc thời gian không xác định.

Thành ngữ 'Break a leg' có nghĩa là 'Chúc may mắn', thường dùng trước khi ai đó lên sân khấu biểu diễn.Sự khác biệt giữa "Affect" và 

KeyboardInterrupt: Interrupted by user

In [ ]:
def run_chat(user_input):
    print(f"\nUser: {user_input}")
    print("-" * 100)
    response = agent_executor.invoke({"input": user_input})
    print(f"Agent: {response['output']}")

# Scenario 1: Sửa lỗi ngữ pháp -> Sẽ gọi 'grammar_corrector'
# run_chat("Kiểm tra ngữ pháp cho câu sau: Yesterday I goed to the store very happy.")
# run_chat("Câu này đúng chưa: She don't like coffee.")
# run_chat("Nhờ bạn chỉnh sửa giúp câu này: He is more taller than me.")
# run_chat("Tôi muốn sửa lỗi ngữ pháp cho: I have been to London last year.")
# run_chat("Chỉnh sửa câu: There is many books on the table.")
# run_chat("Bạn có thể sửa ngữ pháp cho câu: Can you advice me on this matter?")
# run_chat("Xem giúp tôi câu này: I'm interesting in learning English.")

# Scenario 2: Hỏi kiến thức (RAG) -> Sẽ gọi 'search_english_knowledge'
# run_chat("Affect và Effect khác nhau như thế nào? Hãy giải thích chi tiết nhất có thể")
# run_chat("Thành ngữ 'Break a leg' có ý nghĩa gì?")
# run_chat("Khi nào thì dùng thì hiện tại hoàn thành?")
# run_chat("Sự khác biệt giữa 'Colour' và 'Color' là gì?")
# run_chat("Tôi muốn biết thêm về cách dùng của Present Perfect.")

# Scenario 3: Xin lộ trình học -> Sẽ gọi 'learning_suggestion_generator'
# run_chat("Tôi đang ở trình độ B1, tôi muốn học để thi IELTS 7.0. Hãy cho tôi lời khuyên.")

In [4]:
# Scenario 1: Sửa lỗi ngữ pháp -> Sẽ gọi 'grammar_corrector'
# run_chat("Kiểm tra ngữ pháp cho câu sau: Yesterday I goed to the store very happy.")
# run_chat("Câu này đúng chưa: She don't like coffee.")
# run_chat("Nhờ bạn chỉnh sửa giúp câu này: He is more taller than me.")
run_chat("Tôi muốn sửa lỗi ngữ pháp cho: I have been to London last year.")
# run_chat("Chỉnh sửa câu: There is many books on the table.")
# run_chat("Bạn có thể sửa ngữ pháp cho câu: Can you advice me on this matter?")
# run_chat("Xem giúp tôi câu này: Can you advise me on this matter?")
# run_chat("Xem giúp tôi câu này: I'm interesting in learning English.")


# Scenario 2: Hỏi kiến thức (RAG) -> Sẽ gọi 'search_english_knowledge'
# run_chat("Câu 'I'll be mother' có nghĩa gì?")
# run_chat("Affect và Effect khác nhau như thế nào? Hãy giải thích chi tiết nhất có thể")
# run_chat("Thành ngữ 'Break a leg' có ý nghĩa gì?")
# run_chat("Khi nào thì dùng thì hiện tại hoàn thành?")
# run_chat("Sự khác biệt giữa 'Colour' và 'Color' là gì?")
# run_chat("Tôi muốn biết thêm về cách dùng của Present Perfect.")


# Scenario 3: Xin lộ trình học -> Sẽ gọi 'learning_suggestion_generator'
run_chat("Tôi đang ở trình độ A2. Hãy cho tôi lời khuyên.")
# run_chat("Tôi đang ở trình độ B1, tôi muốn học để thi IELTS 7.0. Hãy cho tôi lời khuyên.")
# run_chat("Tôi trình độ A2, mục tiêu là giao tiếp tự tin.")
# run_chat("Tôi là người mới bắt đầu, muốn đọc sách tiếng Anh không cần từ điển.")
# run_chat("Tôi có B2, muốn cải thiện kỹ năng thuyết trình bằng tiếng Anh.")
# run_chat("Tôi đang ở C1, muốn viết luận văn chuyên sâu.")
# run_chat("Tôi cần học tiếng Anh để du lịch, tôi là người mới học.")


User: Tôi muốn sửa lỗi ngữ pháp cho: I have been to London last year.
----------------------------------------------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `grammar_corrector` with `{'text_to_correct': 'I have been to London last year.'}`



    [CORRECTION REPORT]
    Original: I have been to London last year.
    Corrected: I went to London last year.
    Explanation: The phrase "have been" is used for actions that have relevance to the present or for experiences up to now, while "went" is the simple past tense used for actions completed at a specific time in the past. Since "last year" indicates a specific time, the simple past tense is appropriate.
    Câu đã sửa là: **I went to London last year.**

**Giải thích:** Cụm từ "have been" được sử dụng cho các hành động có liên quan đến hiện tại hoặc cho những trải nghiệm đến thời điểm hiện tại, trong khi "went" là thì quá khứ đơn được sử dụng cho các hành động đã hoàn th

In [ ]:
# Scenario 3: Xin lộ trình học -> Sẽ gọi 'learning_suggestion_generator'
# run_chat("Tôi đang ở trình độ B1, tôi muốn học để thi IELTS 7.0. Hãy cho tôi lời khuyên.")
# run_chat("Tôi đang ở trình độ A2. Hãy cho tôi lời khuyên.") # Previous test case
# run_chat("Tôi trình độ A2, mục tiêu là giao tiếp tự tin.")
# run_chat("Tôi là người mới bắt đầu, muốn đọc sách tiếng Anh không cần từ điển.")
# run_chat("Tôi có B2, muốn cải thiện kỹ năng thuyết trình bằng tiếng Anh.")
# run_chat("Tôi đang ở C1, muốn viết luận văn chuyên sâu.")
# run_chat("Tôi cần học tiếng Anh để du lịch, tôi là người mới học.")

# Test cases for the updated tool
run_chat("Tôi muốn nhận lời khuyên học tiếng Anh.") # Missing both
run_chat("Tôi đang ở trình độ B1. Cho tôi lời khuyên.") # Missing goal
run_chat("Mục tiêu của tôi là IELTS 7.0. Cho tôi lời khuyên.") # Missing level


User: Tôi muốn nhận lời khuyên học tiếng Anh.
----------------------------------------------------------------------------------------------------


> Entering new AgentExecutor chain...
Bạn có thể cho tôi biết trình độ hiện tại của bạn (ví dụ: Beginner, A1, B1, C1) và mục tiêu học tập của bạn (ví dụ: IELTS 7.0, giao tiếp tự tin, đọc sách không cần từ điển) không? Điều này sẽ giúp tôi đưa ra lời khuyên phù hợp hơn.

> Finished chain.
Agent: Bạn có thể cho tôi biết trình độ hiện tại của bạn (ví dụ: Beginner, A1, B1, C1) và mục tiêu học tập của bạn (ví dụ: IELTS 7.0, giao tiếp tự tin, đọc sách không cần từ điển) không? Điều này sẽ giúp tôi đưa ra lời khuyên phù hợp hơn.

User: Tôi đang ở trình độ B1. Cho tôi lời khuyên.
----------------------------------------------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `learning_suggestion_generator` with `{'current_level': 'B1'}`


Với trình độ 'B1' của bạn, bạn muốn đạt được mục tiêu 

In [ ]:
run_chat("Tôi cần học tiếng Anh để du lịch, tôi là người mới học.")


User: Tôi cần học tiếng Anh để du lịch, tôi là người mới học.
----------------------------------------------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `learning_suggestion_generator` with `{'current_level': 'Beginner', 'goal': 'du lịch'}`


Dựa trên trình độ Beginner và mục tiêu du lịch, đây là lộ trình gợi ý:
- Tìm kiếm tài liệu và khóa học phù hợp với mục tiêu của bạn.
- Luyện tập thường xuyên các kỹ năng (Nghe, Nói, Đọc, Viết) để nâng cao trình độ.Dựa trên trình độ Beginner và mục tiêu du lịch, đây là lộ trình gợi ý cho bạn:

1. **Tìm kiếm tài liệu và khóa học**: Chọn các khóa học tiếng Anh dành cho người mới bắt đầu, đặc biệt là những khóa học tập trung vào từ vựng và cụm từ liên quan đến du lịch.

2. **Luyện tập thường xuyên**: Cố gắng luyện tập các kỹ năng (Nghe, Nói, Đọc, Viết) hàng ngày. Bạn có thể nghe các đoạn hội thoại tiếng Anh về du lịch, thực hành nói với bạn bè hoặc qua các ứng dụng học tiếng Anh.

3. **Học 